<a href="https://www.kaggle.com/code/nathfavour/kaggle-openai?scriptVersionId=247942053" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>